In [ ]:
"""
SHERPA is a Python library for hyperparameter tuning of machine learning models.
Copyright (C) 2018  Lars Hertel, Peter Sadowski, and Julian Collado.

This file is part of SHERPA.

SHERPA is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

SHERPA is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with SHERPA.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function
import sherpa
from sherpa.algorithms import Genetic
import time
from keras.datasets import mnist
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
batch_size=64
num_classes = 10
epochs = 15

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


In [ ]:
parameters = [sherpa.Discrete('hidden_size', [16, 512]),
              sherpa.Discrete('n_layers', [1, 10]),
              sherpa.Choice('activation', [F.relu, F.tanh, F.sigmoid]),
              sherpa.Continuous('lr',[1e-4,1e-2]),
              sherpa.Continuous('dropout',[0.0,1.0])]
              

In [ ]:
algorithm= Genetic(max_num_trials=100)

In [ ]:
class MLP(nn.Module):
    def __init__(self,in_size,out_size,n_layers,hidden_size,act,dropout):
        super(MLP,self).__init__()
        self.n_layers=n_layers
        self.act=act
        for i in range(n_layers):
            if i==0:
                layer_in_size=in_size
            else:
                layer_in_size=hidden_size
            if i==(n_layers-1):
                layer_out_size=out_size
            else:
                layer_out_size=hidden_size
            
            setattr(self,'dense_{}'.format(i),nn.Linear(layer_in_size,layer_out_size))
            
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,x):
        out=x
        for i in range(self.n_layers):
            if i==(self.n_layers-1):
                out=getattr(self,'dense_{}'.format(i))(self.dropout(out))
            else:
                out=self.act(getattr(self,'dense_{}'.format(i))(self.dropout(out)))
        return out

In [ ]:
train_data = DataLoader(TensorDataset(torch.from_numpy(x_train),torch.from_numpy(y_train).type(torch.long)),batch_size=batch_size,drop_last=True)
criterion = nn.CrossEntropyLoss()
x_test_tensor=torch.from_numpy(x_test)
y_test_tensor=torch.from_numpy(y_test).type(torch.long)

In [ ]:
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=False)
for trial in study:
    print("Trial {}:\t{}".format(trial.id, trial.parameters))
    mlp=MLP(x_train.shape[1],num_classes,
            trial.parameters['n_layers'],
            trial.parameters['hidden_size'],
            trial.parameters['activation'],
            trial.parameters['dropout'])
    mlp.train()
    optimizer=optim.Adam(mlp.parameters(), lr=trial.parameters['lr'])
    for i in range(epochs):
        for x_batch, y_batch in train_data:
            optimizer.zero_grad()
            out=mlp(x_batch)
            loss=criterion(out,y_batch)
            loss.backward()
            optimizer.step()
        mlp.eval()        
        val_acc=(mlp(x_test_tensor).argmax(dim=1)==y_test_tensor).type(torch.float32).mean().item()
        print('val_acc: {}'.format(val_acc))
        study.add_observation(trial=trial,
                          iteration=epochs,
                          objective=val_acc)
        if study.should_trial_stop(trial):
            break 
    study.finalize(trial=trial)

In [ ]:
print(study.get_best_result())